In [15]:
# Necessary Import Statements
import pandas as pd
import numpy as np

# Imports the datasets
data = pd.read_csv('Exoplanet_Data.csv', low_memory=False)
phl = pd.read_csv('PHL_Dataset.csv', low_memory=False)


In [16]:
# Removes any planets that have an NA eccentricity value
data = data[data['pl_orbeccen'].notna()]
data = data[data['pl_rade'].notna()]

# Resets the data row index
data = data.reset_index (drop = True)

In [17]:
# Makes an empty m_type dataframe with same column headers
m_type = pd.DataFrame(columns=list(data.columns))

# Makes an empty m_type_habitable dataframe to store planets that result out of this code
m_type_habitable = pd.DataFrame(columns=list(data.columns))

# Fills the m_type dataframe with M-type host stars' planets
m_type = data.loc[(data['st_spectype'].str.startswith('M', na=False))]

# Resets the m_type row index
m_type = m_type.reset_index (drop = True)

In [18]:
# Takes out planets that have no planet radius and eccentricity
phl = phl[phl['pl_rade'].notna()]
phl = phl[phl['pl_orbeccen'].notna()]

# Resets index
phl = phl.reset_index (drop = True)

In [19]:
# Makes a new empty dataframe for the m-type planets in the PHL dataset
phl_m_type = pd.DataFrame(columns=list(phl.columns))

In [20]:
# Fills the empty dataframe with m-type planets from the PHL dataset
phl_m_type = phl.loc[(phl['st_spectype'].str.startswith('M', na=False))]

# Resets index
phl_m_type = phl_m_type.reset_index (drop = True)

In [21]:
def check_match(planet, control_planet):
    if pd.isna(planet['pl_dens']):
        if (not pd.isna(planet['pl_radj'])) & (not pd.isna(planet['pl_massj'])):
            planet['pl_dens'] = (planet['pl_massj'] * (1.898 * (10 ** 27) * 1000)) \
            / (4 * np.pi * ((planet['pl_radj'] * 43441 * 160934) ** 3) / 3)
        else:
            return
        
    if pd.isna(planet['pl_orbsmax']) and (pd.isna(planet['pl_ratdor']) or pd.isna(planet['st_rad'])):
        return

    # Calculates orbital period in days for main planet
    if pd.isna(planet['pl_orbper']):
        if pd.isna(planet['pl_ratdor']):
            planet['pl_orbper'] = np.sqrt(planet['pl_orbsmax']**3) * 365
        elif not pd.isna(planet['st_rad']): 
            planet['pl_orbper'] = np.sqrt((planet['pl_ratdor']*planet['st_rad'])**3) * 365
        if pd.isna(planet['pl_orbper']):
            return
    
    if pd.isna(control_planet['pl_orbsmax']) and (pd.isna(control_planet['pl_ratdor']) or pd.isna(control_planet['st_rad'])):
        return

    # Calculates orbital period in days for phl planet
    if pd.isna(control_planet['pl_orbper']):
        if pd.isna(control_planet['pl_ratdor']):
            control_planet['pl_orbper'] = np.sqrt(control_planet['pl_orbsmax']**3) * 365
        elif not pd.isna(control_planet['st_rad']): 
            control_planet['pl_orbper'] = np.sqrt((control_planet['pl_ratdor']*control_planet['st_rad'])**3) * 365
        if pd.isna(control_planet['pl_orbper']):
            return
    
    if pd.isna(planet['pl_orbeccen']):
        return
    
    global x
    x = x.append(planet, ignore_index = True)

    global a
    a = a.append(control_planet, ignore_index = True)
    

    if (float(planet['pl_dens']) >= 4) & (float(planet['pl_dens']) <= 7):
        if (planet['pl_controv_flag'] == '0'):
            if ('Transit' in planet['discoverymethod']):
                if (planet['pl_orbeccen'] <= 0.2):
                    if (np.abs(float(planet['pl_orbper']) - float(control_planet['pl_orbper'])) <= 50):
                        global m_type_habitable
                        m_type_habitable = m_type_habitable.append(planet, ignore_index = True)


In [22]:
closest_distance_away = 10000
index = -1

for i in range (len(m_type['pl_rade'])): 
    current_dist = np.abs(phl_m_type['pl_rade'][0]-m_type['pl_rade'][i])
    if current_dist < 10:
        closest_distance_away = current_dist
        index = 0
        
    current_dist = np.abs(phl_m_type['pl_rade'][1]-m_type['pl_rade'][i])
    if current_dist < 10:
        if closest_distance_away > current_dist:
            closest_distance_away = current_dist
            index = 1
            
    current_dist = np.abs(phl_m_type['pl_rade'][2]-m_type['pl_rade'][i])
    if current_dist < 10:
        if closest_distance_away > current_dist:
            closest_distance_away = current_dist
            index = 2
            
    if index == -1 or closest_distance_away == 10000:
        continue
        
    planet = m_type.loc[i].copy()
    control_planet = phl_m_type.loc[index].copy()
    
    check_match(planet, control_planet)
    closest_distance_away = 10000
    index = -1



In [23]:
m_type_habitable

,rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,default_flag,sy_snum,...,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,Unnamed: 287,Unnamed: 288,Unnamed: 289,Unnamed: 290,Unnamed: 291,Unnamed: 292
0,359,GJ 1132 b,GJ 1132,b,NaN,NaN,TIC 101955023,Gaia DR2 5413438219396893568,0,1,...,0,0,0,23,NaN,NaN,NaN,NaN,NaN,NaN
1,27884,L 98-59 c,L 98-59,c,NaN,NaN,TIC 307210830,Gaia DR2 5271055243163629056,1,1,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,27905,LTT 3780 b,LTT 3780,b,NaN,NaN,TIC 36724087,Gaia DR2 3767281845873242112,1,2,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,28367,TOI-1266 b,TOI-1266,b,NaN,NaN,TIC 467179528,Gaia DR2 1678074272650459008,1,1,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
